In [2]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset


 95% 313M/330M [00:02<00:00, 128MB/s]
100% 330M/330M [00:02<00:00, 132MB/s]


In [4]:
!unzip -qq face-mask-12k-images-dataset.zip

In [1]:
cd /content/drive/MyDrive/DeepLearningTasks/FaceMaskDetection

/content/drive/MyDrive/DeepLearningTasks/FaceMaskDetection


In [2]:
%%writefile config.py

batch_size = 32
epochs = 5
lr = 0.0001
width = height = 128

Overwriting config.py


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from config import *

In [4]:
data_generator = ImageDataGenerator(
    rescale = 1/.255,
    horizontal_flip=True
)

train_data = data_generator.flow_from_directory(
    "/content/Face Mask Dataset/Train",
    target_size = (width, height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    "/content/Face Mask Dataset/Validation",
    target_size = (width, height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True,
)

print(np.bincount(train_data.labels))
print(np.bincount(val_data.labels))

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
[5000 5000]
[400 400]


In [5]:
model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
])

In [6]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                             metrics=['accuracy'])

In [7]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          batch_size=batch_size,
          epochs=epochs,
          )

Epoch 1/5
312/312 [==============================] - 915s 3s/step - loss: 0.1293 - accuracy: 0.9663 - val_loss: 0.0441 - val_accuracy: 0.9862
Epoch 2/5
312/312 [==============================] - 935s 3s/step - loss: 0.0376 - accuracy: 0.9886 - val_loss: 0.0236 - val_accuracy: 0.9912
Epoch 3/5
312/312 [==============================] - 923s 3s/step - loss: 0.0207 - accuracy: 0.9942 - val_loss: 0.1194 - val_accuracy: 0.9550
Epoch 4/5
312/312 [==============================] - 911s 3s/step - loss: 0.0163 - accuracy: 0.9947 - val_loss: 0.0702 - val_accuracy: 0.9775
Epoch 5/5
312/312 [==============================] - 931s 3s/step - loss: 0.0184 - accuracy: 0.9950 - val_loss: 0.0318 - val_accuracy: 0.9900


In [8]:
model.save("/content/drive/MyDrive/DeepLearningTasks/FaceMaskDetection/model.h5")

In [9]:
test_data_path = '/content/Face Mask Dataset/Test'
model = load_model('/content/drive/MyDrive/DeepLearningTasks/FaceMaskDetection/model.h5')

data_generator = ImageDataGenerator(rescale = 1/.255)

test_data = data_generator.flow_from_directory(
    "/content/Face Mask Dataset/Test",
    target_size = (width, height),
    batch_size=batch_size,
    class_mode = 'categorical',
    shuffle = False,
)

model.evaluate(test_data)

Found 992 images belonging to 2 classes.
31/31 [==============================] - 22s 710ms/step - loss: 0.0402 - accuracy: 0.9839


[0.04020189493894577, 0.9838709831237793]

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[475   8]
 [  8 501]]
Classification Report
              precision    recall  f1-score   support

    WithMask       0.98      0.98      0.98       483
 WithoutMask       0.98      0.98      0.98       509

    accuracy                           0.98       992
   macro avg       0.98      0.98      0.98       992
weighted avg       0.98      0.98      0.98       992

